# Creating a pure kinematic bioMod model
This file shows how to create a `.bioMod` file, what should be declared, what it means, etc.
Additionnally, we explore how to generate the file programmatically. 
Please note that nothing prevents you from writting the file by hand.

In a nutshell a `.bioMod` file is a description of a rigid body kinematic chain. 
It therefore expects a serie of segments linked by degree of freedom, up to six (up to three translations and three rotations). 
Once loaded, if a degree of freedom is given to the model, then one can interact with it on this degree of freedom.

If one is interested only by the kinematics, then the inertial parameters can be omitted (which we omit here). 
Otherwise, they have to be declared (see `2-dynamic_model_creation`).

*Please note, an easier way to generate a new model would be to use `biobuddy`, a Python package that is designed for creating and translating biomechanical models. However in order to show a complete and self-contained example, we will create the model manually in this tutorial.*

## Make our life easier
First of all, let's make our life easier by defining a class that we will fill for each of the segments and markers.

This section helps automatically write the file. 
If you want to write it by hand, you can skip this section.

In [ ]:
import os 

# Prepare the name of the output file and create a 'models' folder if needed
kinematic_model_file_path = "models/SimpleBody.bioMod"
kinematic_chain_save_path_file = "models/simple_body_kinematic_chain.kc"

if not os.path.isdir("models"):
    os.mkdir("models")

In [ ]:
# First of all, let's make our life easier by defining a class that we will fill for each of the segment
# Notice that mass, center_of_mass and inertia_xxyyzz are currently ignored. They will become relevant in 
# dynamics section below
class Segment():
    def __init__(
        self, 
        name, 
        parent_name="", 
        rt="",
        translations="", 
        rotations="",
        mass=0,
        center_of_mass=None,
        inertia_xxyyzz=None,
        mesh: tuple=None,
        markers: tuple=None,
    ):
        self.name = name
        self.parent_name = parent_name
        self.rt = rt
        self.translations = translations
        self.rotations = rotations
        self.mass = mass
        self.center_of_mass=center_of_mass
        self.inertia_xxyyzz=inertia_xxyyzz
        self.mesh = mesh
        self.markers = markers

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"segment {self.name}\n"
        if (self.parent_name):
            out_string += f"\tparent {self.parent_name}\n"
        if (self.rt):
            out_string += f"\tRT {self.rt}\n"
        if (self.translations):
            out_string += f"\ttranslations {self.translations}\n"
        if (self.rotations):
            out_string += f"\trotations {self.rotations}\n"
        if (self.mass):
            out_string += f"\tmass {self.mass}\n"
        if (self.center_of_mass):
            out_string += f"\tcom {self.center_of_mass[0]} {self.center_of_mass[1]} {self.center_of_mass[2]}\n"
        if (self.inertia_xxyyzz):
            out_string += f"\tinertia {self.inertia_xxyyzz[0]} 0 0\n" + \
                          f"\t        0 {self.inertia_xxyyzz[1]} 0\n" + \
                          f"\t        0 0 {self.inertia_xxyyzz[2]}\n"
        if (self.mesh):
            for m in self.mesh:
                out_string += f"\tmesh {m[0]} {m[1]} {m[2]}\n"
        out_string += "endsegment\n"
        
        # Also print the markers attached to the segment
        if (self.markers):
            for marker in self.markers:
                out_string += str(marker)
        return out_string


# Here is an example on how to use this class
my_new_segment = Segment(
    name="my_new_semgent_name", 
    parent_name="the_name_of_the_parent_segment", 
    rt="0 0 0 xyz 0 0 0",  # The transformation between the segment and its parent. The first three values are the three rotations about the given sequence (here xyz) and then the three translations
    translations="yxz",  # The degrees of fredom in translation
    rotations="xyz",  # The degrees of fredom in rotation
    mesh=((0, 0, 0), (0, 0, 0.3))  # The list of mesh point to show to create the stick figure. All the points are link in order
)
print(my_new_segment)  # Later this print should be sent to a file

In [ ]:
# Let's do the same for the markers
class Marker():
    def __init__(
        self, 
        name, 
        parent_name, 
        position,
    ):
        self.name = name
        self.parent_name = parent_name
        self.position = position

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"marker {self.name}\n"
        out_string += f"\tparent {self.parent_name}\n"
        out_string += f"\tposition {self.position[0]} {self.position[1]} {self.position[2]}\n"
        out_string += "endmarker\n"
        return out_string

# Here is an example on how to use this class
my_new_marker = Marker(
    name="my_new_marker_name",
    parent_name="the_name_of_the_parent_segment",
    position=(0, 0, 1),
)
print(my_new_marker)  # Later this print should be sent to a file

In [ ]:
# And why not creating a data structure that will collect all the segments and produce the a full
# kinematic chain that can easily be printed
class KinematicChain():
    def __init__(self, segments):
        self.segments = segments
        
    def __str__(self):
        out_string = "version 4\n\n"
        for segment in self.segments:
            out_string += str(segment)
            out_string += "\n\n\n"  # Give some space between segments
        return out_string
    
    def write(self, file_path):
        # Method to write the current KinematicChain to a file
        with open(file_path, "w") as file:
            file.write(str(self))

## Create the `SimpleBody.bioMod` kinematic chain
Now we are ready to produce the kinematic chain.
It is a 2D model made from the mean dimension of the adjusted De Leva model (see the doc folder):
- A TRUNK segment that includes the HEAD (up to the ear) at $72cm$ that moves frontward and upward (translations y and z) and rotations about the medio-lateral axis (rotations x);
- An ARM segment attached to the TRUNK at $53cm$ that includes the HAND at $61cm$. It rotates about the medio-lateral axis (rotations x);
- An THIGH segment attached to the TRUNK. It rotates about the medio-lateral axis (rotations x)
- An SHANK segment attached to the THIGH at $42cm$. It rotates about the medio-lateral axis (rotations x)
- An FOOT segment attached to the SHANK at $43cm$ that includes a FOOT of $25cm$. It rotates about the medio-lateral axis (rotations x)

In [ ]:
# Let's define the segments and the skin markers attached to each segment 
# and collect all the values in the KinematicChain class

# The trunk segment
trunk_marker_pelvis = Marker(
    name="PELVIS", 
    parent_name="TRUNK",
    position=(0, 0, 0),
)
trunk = Segment(
    name="TRUNK",
    translations="yz", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, 0.53)),
    markers=[trunk_marker_pelvis]
)



# The head segment
top_head_marker_head = Marker(
    name="TOP_HEAD",
    parent_name="HEAD",
    position=(0, 0, 0.24),
)
head = Segment(
    name="HEAD",
    parent_name="TRUNK",
    rt="0 0 0 xyz 0 0 0.53",
    mesh=((0, 0, 0), (0, 0, 0.24)),
    markers=[top_head_marker_head],
)



# The arm segment
shoulder_marker = Marker(
    name="SHOULDER", 
    parent_name="UPPER_ARM",
    position=(0, 0, 0),
)
upper_arm = Segment(
    name="UPPER_ARM",
    parent_name=trunk.name, 
    rt="0 0 0 xyz 0 0 0.53", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.28)),
    markers=[shoulder_marker],
)

elbow_marker = Marker(
    name="ELBOW",
    parent_name="LOWER_ARM",
    position=(0, 0, 0),
)
lower_arm = Segment(
    name="LOWER_ARM",
    parent_name=upper_arm.name,
    rt="0 0 0 xyz 0 0 -0.28",
    mesh=((0, 0, 0), (0, 0, -0.27)),
    markers=[elbow_marker],
)

wrist_marker = Marker(
    name="WRIST",
    parent_name="HAND",
    position=(0, 0, 0),
)
finger_marker = Marker(
    name="FINGER",
    parent_name="HAND",
    position=(0, 0, -0.19),
)
hand = Segment(
    name="HAND",
    parent_name=lower_arm.name,
    rt="0 0 0 xyz 0 0 -0.27",
    mesh=((0, 0, 0), (0, 0, -0.19)),
    markers=[wrist_marker, finger_marker]
)



# The thigh segment
thigh = Segment(
    name="THIGH", 
    parent_name=trunk.name, 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.42)),
)


# The shank segment
knee_marker = Marker(
    name="KNEE", 
    parent_name="SHANK",
    position=(0, 0, 0),
)
shank = Segment(
    name="SHANK", 
    parent_name=thigh.name, 
    rt="0 0 0 xyz 0 0 -0.42", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.43)),
    markers=[knee_marker],
)


# The foot segment
ankle_marker = Marker(
    name="ANKLE", 
    parent_name="FOOT",
    position=(0, 0, 0),
)
toe_marker = Marker(
    name="TOE", 
    parent_name="FOOT",
    position=(0, 0, 0.25),
)
foot = Segment(
    name="FOOT", 
    parent_name=shank.name, 
    rt="0 0 0 xyz 0 0 -0.43", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, 0.25)),
    markers=[ankle_marker, toe_marker],
)

# Put the model together, print it and print it to a bioMod file
kinematic_chain = KinematicChain(segments=(trunk, head, upper_arm, lower_arm, hand, thigh, shank, foot))
print(kinematic_chain)
kinematic_chain.write(kinematic_model_file_path)

# Let's also save the kinematic_chain as a python structure. This will be useful in `2-DynamicModelCreation`
import pickle
with open(kinematic_chain_save_path_file, "wb") as file:
    pickle.dump(kinematic_chain, file)

In [ ]:
#
# THIS CELL IS COMMENTED AS IT CANNOT BE RUN FROM THE BINDER
# If you are running this notebook locally, you can uncomment the cell to visualize the model you just created
# 

# # Run this cell to show the model you just created. 
# import bioviz

# # Send the previously loaded model to the vizualizer
# viz = bioviz.Viz(kinematic_model_file_path)

# # Move the model to a recognizable position (arm raised and knee flexed)
# viz.set_q((0, 0, -0.15, 1.20, 0.7, -1, -1.11))

# # Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
# viz.exec()

# # If nothing happens, check for background. Sometimes the window loads behind the current window